In [1]:
from package import process_dataset
from package import encoder
from package import models

import numpy as np

from sklearn.model_selection import train_test_split
import pandas as pd

Loaded  7 encoders.


/home/gustavo/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/gustavo/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
description = pd.read_csv("data/dicionario.csv", encoding='latin-1')
train = pd.read_csv("data/dados_treino_hackaton.csv", encoding='latin-1', index_col=False)
test = pd.read_csv("data/dados_teste_x_hackaton.csv", encoding='latin-1', index_col=False)

del train['Unnamed: 0']
del test['Unnamed: 0']

In [3]:
train = encoder.encode_DataFrame(train)
train = process_dataset.processColumns(train)

In [4]:
train_cols, target = process_dataset.generate_labels(train)

In [5]:
X = process_dataset.scaleData(train[train_cols])
y = train[target]

Using saved scaler.


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Linear regression

In [7]:
from sklearn.linear_model import LinearRegression

In [8]:
lin = LinearRegression()
lin.fit(X_train, y_train)

LinearRegression()

In [9]:
pred = lin.predict(X_test)

In [10]:
ypred = models.convertPredicted(pred)
models.run_analysis(ypred, y_test)

              precision    recall  f1-score   support

           0       0.73      0.75      0.74        63
           1       0.73      0.72      0.72        60

    accuracy                           0.73       123
   macro avg       0.73      0.73      0.73       123
weighted avg       0.73      0.73      0.73       123

Acc score:  0.7317073170731707
Mean Squared Error:  0.2682926829268293





## Decision tree regressor

In [11]:
from sklearn.tree import DecisionTreeRegressor

In [12]:
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)

DecisionTreeRegressor()

In [13]:
models.run_analysis(tree.predict(X_test), y_test)

              precision    recall  f1-score   support

           0       0.59      0.60      0.60        63
           1       0.58      0.57      0.57        60

    accuracy                           0.59       123
   macro avg       0.59      0.58      0.58       123
weighted avg       0.59      0.59      0.59       123

Acc score:  0.5853658536585366
Mean Squared Error:  0.4146341463414634





## Random Forest Regressor

In [14]:
from sklearn.ensemble import RandomForestRegressor

In [15]:
forest = RandomForestRegressor()
forest.fit(X_train, y_train)

RandomForestRegressor()

In [16]:
ypred = models.convertPredicted(forest.predict(X_test))

In [17]:
models.run_analysis(ypred, y_test)

              precision    recall  f1-score   support

           0       0.74      0.76      0.75        63
           1       0.74      0.72      0.73        60

    accuracy                           0.74       123
   macro avg       0.74      0.74      0.74       123
weighted avg       0.74      0.74      0.74       123

Acc score:  0.7398373983739838
Mean Squared Error:  0.2601626016260163





## Testing polynomial regressor

In [18]:
from sklearn.preprocessing import PolynomialFeatures

In [23]:
p = PolynomialFeatures(degree = 3, include_bias=False)

In [24]:
pX_train = p.fit_transform(X_train)

In [25]:
lin = LinearRegression()
lin.fit(pX_train, y_train)

LinearRegression()

In [26]:
pred = lin.predict(p.transform(X_test))
ypred = models.convertPredicted(pred)
models.run_analysis(ypred, y_test)

              precision    recall  f1-score   support

           0       0.61      0.60      0.61        63
           1       0.59      0.60      0.60        60

    accuracy                           0.60       123
   macro avg       0.60      0.60      0.60       123
weighted avg       0.60      0.60      0.60       123

Acc score:  0.6016260162601627
Mean Squared Error:  0.3983739837398374





## SGD Regression

In [31]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error

from sklearn.base import clone

In [32]:
sgd = SGDRegressor(max_iter=1, tol=-np.infty, warm_start=True, penalty=None, learning_rate="constant", eta0=0.0005)

In [59]:
minimal_error = float("inf")
best_epoch = None
best_model = None

for epoch in range(6000):
    sgd.fit(X_train, y_train)
    y_val_pred = sgd.predict(X_test)
    val_error = mean_squared_error(y_test, y_val_pred)
    if val_error < minimal_error:
        minimal_error = val_error
        best_epoch = epoch
        best_model = clone(sgd)


In [60]:
ypred = models.convertPredicted(sgd.predict(X_test))
models.run_analysis(ypred, y_test)

              precision    recall  f1-score   support

           0       0.72      0.75      0.73        63
           1       0.72      0.70      0.71        60

    accuracy                           0.72       123
   macro avg       0.72      0.72      0.72       123
weighted avg       0.72      0.72      0.72       123

Acc score:  0.7235772357723578
Mean Squared Error:  0.2764227642276423





## Logistic Regression

In [62]:
from sklearn.linear_model import LogisticRegression

In [63]:
log = LogisticRegression()

In [64]:
log.fit(X_train, y_train)

LogisticRegression()

In [66]:
ypred = log.predict(X_test)
models.run_analysis(ypred, y_test)

              precision    recall  f1-score   support

           0       0.70      0.73      0.71        63
           1       0.70      0.67      0.68        60

    accuracy                           0.70       123
   macro avg       0.70      0.70      0.70       123
weighted avg       0.70      0.70      0.70       123

Acc score:  0.6991869918699187
Mean Squared Error:  0.3008130081300813





## Support vector machines

In [75]:
from sklearn.svm import LinearSVC

In [76]:
svm = LinearSVC()
svm.fit(X_train, y_train)

LinearSVC()

In [77]:
ypred = svm.predict(X_test)
models.run_analysis(ypred, y_test)

              precision    recall  f1-score   support

           0       0.72      0.73      0.72        63
           1       0.71      0.70      0.71        60

    accuracy                           0.72       123
   macro avg       0.72      0.72      0.72       123
weighted avg       0.72      0.72      0.72       123

Acc score:  0.7154471544715447
Mean Squared Error:  0.2845528455284553





In [79]:
from sklearn.svm import SVC

In [94]:
svm = SVC(kernel='poly', C=10)
svm.fit(X_train, y_train)

SVC(C=10, kernel='poly')

In [95]:
ypred = svm.predict(X_test)
models.run_analysis(ypred, y_test)

              precision    recall  f1-score   support

           0       0.67      0.79      0.72        63
           1       0.73      0.58      0.65        60

    accuracy                           0.69       123
   macro avg       0.70      0.69      0.69       123
weighted avg       0.70      0.69      0.69       123

Acc score:  0.6910569105691057
Mean Squared Error:  0.3089430894308943





## Decision tr

# Fine Tuning

In [139]:
from sklearn.model_selection import GridSearchCV

In [147]:
param_grid = [
    {'n_estimators': [3, 7, 10, 20, 30], 'max_features': [2, 4, 6, 8, 10]},
    {'bootstrap': [False], 'n_estimators': [3, 7, 10, 12], 'max_features': [2, 3, 4, 8, 12, 15]},
]

In [148]:
forest = RandomForestRegressor()

In [149]:
grid = GridSearchCV(forest, param_grid, cv=5, scoring="neg_mean_squared_error", return_train_score=True, refit=True)

In [150]:
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8, 10],
                          'n_estimators': [3, 7, 10, 20, 30]},
                         {'bootstrap': [False],
                          'max_features': [2, 3, 4, 8, 12, 15],
                          'n_estimators': [3, 7, 10, 12]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [151]:
grid.best_params_

{'max_features': 2, 'n_estimators': 30}

In [152]:
ypred = models.convertPredicted(grid.predict(X_test))

In [153]:
models.run_analysis(ypred, y_test)

              precision    recall  f1-score   support

           0       0.66      0.81      0.73        63
           1       0.74      0.57      0.64        60

    accuracy                           0.69       123
   macro avg       0.70      0.69      0.69       123
weighted avg       0.70      0.69      0.69       123

Acc score:  0.6910569105691057
Mean Squared Error:  0.3089430894308943





## Importancia de colunas

In [154]:
sorted(zip(grid.best_estimator_.feature_importances_, train_cols))

[(0.0, 'Gestão da Saúde'),
 (0.00027965563996193546, 'Treinamentos'),
 (0.0005254419776481598, 'Cybersecurity'),
 (0.001294800977292973, 'Equilíbrio fiscal'),
 (0.0012948623367975286, 'Comissão sobre Parceiros'),
 (0.0027888095030159776, 'Gestão da Educação'),
 (0.0030011101232366453, 'Concorrentes'),
 (0.0034694937314160234, 'Gestão da Receita'),
 (0.003682558840183551, 'Gestão da Segurança Viária'),
 (0.004070672250684712, 'ESG'),
 (0.005938321763720103, 'Gestão de operações projetizadas'),
 (0.006013639778833222, 'Gestão de Gastos'),
 (0.006032163515221844, 'Skill_dev'),
 (0.006294935522448042, 'Software'),
 (0.007990456408537341, 'Gestão Estratégica'),
 (0.009734575431953156, 'Produtos digitais'),
 (0.010483857132564781, 'Gestão da Receita_per_Gestão de Gastos'),
 (0.01068307970725396, 'Gestão de pessoas'),
 (0.013878403099484508, 'Processes Excellence'),
 (0.01642835372704268, 'n_solucoes'),
 (0.0169633394532104, 'Desdobramento de metas'),
 (0.022790247121722555, 'Gestão da Operaç

## Using cross validation

In [25]:
from sklearn.model_selection import cross_val_score, cross_val_predict

In [20]:
scores = cross_val_score(tree, X, y, scoring="neg_mean_squared_error", cv=10)

In [32]:
display_scores(np.sqrt(-scores))

Scores:  [0.62217102 0.5956834  0.65991202 0.52790958 0.70128687 0.74657689
 0.6401844  0.6140433  0.62725005 0.65286255]
Mean:  0.6387880074544369
Std:  0.05598606940195762


In [22]:
def display_scores(scores):
    print("Scores: ", scores)
    print("Mean: ", scores.mean())
    print("Std: ", scores.std())

In [29]:
models.run_analysis(cross_val_predict(tree, X, y, cv=10), y)

              precision    recall  f1-score   support

           0       0.62      0.59      0.60       326
           1       0.56      0.59      0.57       287

    accuracy                           0.59       613
   macro avg       0.59      0.59      0.59       613
weighted avg       0.59      0.59      0.59       613

Acc score:  0.5889070146818923
Mean Squared Error:  0.4110929853181077



